# Approach for 3D coordinates

In [ ]:
# standard
import h5py

# 3rd party
import numpy as np
import os

# our own 
import skeletalModel_new
import pose2D
import pose2Dto3D
import pose3D
import plot_skeleton_BH

In [ ]:
def convList2Array(lst): 
    T, dim = lst[0].shape
    a = []
    for t in range(T):
        a_t = []
        for i in range(dim):
            for j in range(len(lst)):
                a_t.append(lst[j][t, i])
        a.append(a_t)
    return np.asarray(a)


def use_filter(x):
    structure = skeletalModel_new.getSkeletalModelStructure()

    inputSequence_3D = x

    # Decomposition of the single matrix into four matrices: x, y, z, w (=likelihood)

    X = inputSequence_3D

    Xx = X[:, 0::4]
    Xy = X[:, 1::4]
    Xz = X[:, 2::4]
    Xw = X[:, 3::4]
    
    """
    o = 8383
    xyz_points = np.array([Xx[o], Xy[o], Xz[o], Xw[o]]).transpose()
    xyz_points = xyz_points.flatten()
    plot_skeleton3d_new.plot_3D_skeleton(xyz_points)
    """
    
    # Normalization of the picture (x and y axis has the same scale)
    Xx, Xy, Xz = pose2D.normalization(Xx, Xy, Xz)
    

    # Delete all skeletal models which have a lot of missing parts.
    Xx, Xy, Xz, Xw = pose2D.prune(Xx, Xy, Xz, Xw, (0, 1, 2, 3, 4, 5, 6, 7), 0.3, dtype)
    
    # Preliminary filtering: weighted linear interpolation of missing points.
    Xx, Xy, Xz, Xw = pose2D.interpolation(Xx, Xy, Xz, Xw, 0.99, dtype)
    
    """
    o = 8383
    xyz_points = np.array([Xx[o], Xy[o], Xz[o], Xw[o]]).transpose()
    print(xyz_points.shape)
    xyz_points = xyz_points.flatten()
    print(xyz_points.shape)
    plot_skeleton3d_new.plot_3D_skeleton(xyz_points)
    """
    #print(Xx.shape)
    # Initial 3D pose estimation
    lines0, rootsx0, rootsy0, rootsz0, anglesx0, anglesy0, anglesz0, Yx0, Yy0, Yz0, Yw0 = pose2Dto3D.initialization(
        Xx,
        Xy,
        Xz,
        Xw,
        structure,
        0.001, # weight for adding noise
        randomNubersGenerator,
        dtype,
        percentil=0.5,
    )
    #print(Yx0.shape)
    """
    o = 8383
    xyz_points = np.array([Yx0[o], Yy0[o], Yz0[o], Xw[o]]).transpose()
    xyz_points = xyz_points.flatten()
    plot_skeleton3d_new.plot_3D_skeleton(xyz_points)
    """

    # Backpropagation-based filtering
    Yx, Yy, Yz = pose3D.backpropagationBasedFiltering(
        lines0, 
        rootsx0,
        rootsy0, 
        rootsz0,
        anglesx0,
        anglesy0,
        anglesz0,   
        Xx,   
        Xy,
        Xz,
        Xw,
        structure,
        dtype,
        nCycles=100,
    )
    #print(Yx.shape)
    """
    o = 8383
    xyz_points = np.array([Yx[o], Yy[o], Yz[o], Xw[o]]).transpose()
    print(xyz_points.shape)
    xyz_points = xyz_points.flatten()
    print(xyz_points.shape)
    plot_skeleton3d_new.plot_3D_skeleton(xyz_points)
    """

    return convList2Array([Yx, Yy, Yz])

In [ ]:
if __name__ == "__main__":
    
    
    fnameIn = "../Data/Rotated/"
    npy_files = [file for file in os.listdir(fnameIn) if file[-4:] == '.npy']
    
    fnameOut = "../Data/Preprocessed_CZ/"

    dtype = "float32"
    randomNubersGenerator = np.random.RandomState(1234)
    
    for key in npy_files:
        print("")
        print("... processing '%s'" % key)
        print("")
        with open(fnameIn + key, 'rb') as f:
            x = np.load(f)
        y = use_filter(x)
        with open(fnameOut + key, 'wb') as f:
            np.save(f, y)
        